In [ ]:
!pip install wandb

In [ ]:
import wandb
!wandb login '9a5ccd0848378e80e2abf8c49fbe7f9d7c5e0b10'

In [ ]:
import torch.nn as nn
from torch.nn import functional as funct
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import os
from PIL import Image
from torch import optim
import torch
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# !pip install wget
# import wget
# wget.download('https://storage.googleapis.com/wandb_datasets/nature_12K.zip')
# !unzip /kaggle/working/nature_12K.zip

In [ ]:
class CNN(nn.Module):

  def __init__(self,settings):
    super().__init__()
    input_channel = settings['input_channel']
    output_size = settings['output_size']
    total_filters = settings['total_filters']
    filter_size = settings['filter_size']

    filter_list_out = self.organise_filter_function(settings['filter_organisation'],total_filters)

    stride_filter = settings['stride']
    padding_filter = settings['padding']
    filter_pool = settings['filter_pool']
    padding_pool = settings['padding_pool']
    stride_pool = settings['stride_pool']
    activation = settings['activation']

    activation = self.activation_call(activation)

    dense_layer_size = settings['dense_layer_size']
    image_size = settings['image_size']
    dropout_prob = settings['dropout']
    self.batch_normalisation = settings['batch_normalisation']

# batct normalisation
    if(self.batch_normalisation == 'Yes'):
      self.conv1_bn = nn.BatchNorm2d(filter_list_out[0])
      self.conv2_bn = nn.BatchNorm2d(filter_list_out[1])
      self.conv3_bn = nn.BatchNorm2d(filter_list_out[2])
      self.conv4_bn = nn.BatchNorm2d(filter_list_out[3])
      self.conv5_bn = nn.BatchNorm2d(filter_list_out[4])

# generating convolution layer and pooling layers along with the dense layer with appropriate sizes

    #-------------------------------------------------------------------------------CONV1--------------------------------------------------------------------------------------------

    self.conv1 = nn.Conv2d(in_channels = input_channel, out_channels =filter_list_out[0], kernel_size = filter_size[0], padding = padding_filter, stride = stride_filter)
    self.activation1 = activation
    self.pool1 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size[0] ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #--------------------------------------------------------------------------------CONV2---------------------------------------------------------------------------------------

    self.conv2 = nn.Conv2d(in_channels = filter_list_out[0], out_channels =filter_list_out[1], kernel_size = filter_size[1], padding = padding_filter, stride = stride_filter)
    self.activation2 = activation
    self.pool2 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size[1] ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #---------------------------------------------------------------------------------CONV3------------------------------------------------------------------------------------------

    self.conv3 = nn.Conv2d(in_channels = filter_list_out[1], out_channels =filter_list_out[2], kernel_size = filter_size[2], padding = padding_filter, stride = stride_filter)
    self.activation3 = activation
    self.pool3 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size[2] ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #----------------------------------------------------------------------------------CONV4-----------------------------------------------------------------------------------------

    self.conv4 = nn.Conv2d(in_channels = filter_list_out[2], out_channels =filter_list_out[3], kernel_size = filter_size[3], padding = padding_filter, stride = stride_filter)
    self.activation4 = activation
    self.pool4 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size[3] ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #-----------------------------------------------------------------------------------CONV5----------------------------------------------------------------------------------------

    self.conv5 = nn.Conv2d(in_channels = filter_list_out[3], out_channels = filter_list_out[4], kernel_size = filter_size[4], padding = padding_filter, stride = stride_filter)
    self.activation5 = activation
    self.pool5 = nn.MaxPool2d(kernel_size = filter_pool, padding = padding_pool, stride = stride_pool)

    image_size =  self.compute_conv_size(image_size , filter_size[4] ,stride_filter , padding_filter , 1 , filter_pool, padding_pool, stride_pool)

    #-----------------------------------------------------------------------------------DENSE_LAYER-------------------------------------------------------------------------------------
    self.flatten = nn.Flatten()
    self.dropout = nn.Dropout(dropout_prob)
    self.FC1 = nn.Linear(filter_list_out[4]*image_size*image_size,dense_layer_size)
    self.activationfc1 = activation
    self.op_layer = nn.Linear(dense_layer_size,output_size)

#=============================================================================== Utility Functions ========================================================================================

  def organise_filter_function(self, org_type, filter_size):

    if(org_type == 'same'):
      factor = 1
    if(org_type == 'double'):
      factor = 2
    if(org_type == 'half'):
      factor = 0.5

    filter_list = []
    for i in range(5):
      filter_list.append(filter_size)
      filter_size = int(filter_size*factor)

    return filter_list

  def activation_call(self, act_type):

    if act_type == 'ReLU':
      return nn.ReLU()
    if act_type == 'GELU':
      return nn.GELU()
    if act_type == 'SiLU':
      return nn.SiLU()
    if act_type == 'Mish':
      return nn.Mish()

  def compute_conv_size(self,W, F, S, P, DP, FP, PP, SP):
    W =  (W - F + 2*P)//S + 1
    return (W + 2*PP - (DP* (FP-1)) - 1)//SP + 1



  def forward(self,data):

    data = self.conv1(data)
    if(self.batch_normalisation == 'Yes'):
      data = self.conv1_bn(data)
    data = self.activation1(data)
    data = self.pool1(data)

    data = self.conv2(data)
    if(self.batch_normalisation == 'Yes'):
      data = self.conv2_bn(data)
    data = self.activation2(data)
    data = self.pool2(data)

    data = self.conv3(data)
    if(self.batch_normalisation == 'Yes'):
      data = self.conv3_bn(data)
    data = self.activation3(data)
    data = self.pool3(data)

    data = self.conv4(data)
    if(self.batch_normalisation == 'Yes'):
      data = self.conv4_bn(data)
    data = self.activation4(data)
    data = self.pool4(data)

    data = self.conv5(data)
    if(self.batch_normalisation == 'Yes'):
      data = self.conv5_bn(data)
    data = self.activation5(data)
    data = self.pool5(data)

    data = self.flatten(data)
    data = self.dropout(data)
    data = self.FC1(data)
    data = self.activationfc1(data)
    data = self.op_layer(data)
    data = funct.softmax(data,dim = 1)

    return data




In [ ]:
def dataset_loader(image_size , aug_type , batch_size,split_ratio):

  test_trans = transforms.Compose([transforms.Resize((image_size, image_size)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

  if(aug_type == 'Yes'):

    train_trans = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomRotation(degrees=30),transforms.Resize((image_size, image_size)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

  else:
    train_trans = test_trans

  dataset = "/kaggle/input/naturalistdata/inaturalist_12K"
  train = datasets.ImageFolder(os.path.join(dataset, 'train'), transform = train_trans)
  test = datasets.ImageFolder(os.path.join(dataset, 'val'), transform = test_trans)

  class_set = train.classes

  val = 1999
  tr = 9999-val
  train,validation = random_split(train, [tr, val])

  load_train = DataLoader(train, batch_size = batch_size, num_workers=4)
  load_validation = DataLoader(validation, batch_size = batch_size, num_workers=4)
  load_test = DataLoader(test, batch_size = batch_size, num_workers=4)


  return class_set , load_train , load_test ,load_validation



In [ ]:
sweep_config = {
    'name' : "sweep_30_final",
    'method': 'random'
    }

parameters_sweep = {

  'input_channel' : {'values' : [3] },
  'output_size' : {'values' : [10] },
  'data_augmentation' : {'values' : ['Yes','No'] }, # "Yes", "No"
  'batch_normalisation' : {'values' : ['Yes','No'] }, # "Yes", "No"
  'dropout' : {'values' : [0.2,0.1,0.3] },
  'filter_organisation' : {'values' : ['same','double','half'] }, #'same', 'double', 'half'
  'total_filters': {'values' : [32,64,128] }, #total filters on each layer
  'filter_size' : {'values' : [[7,5,5,3,3],[11,7,5,3,3],[3,3,3,3,3]] }, #each filter size
  'stride' : {'values' : [1,2] }, #stride value for filter
  'padding' : {'values' : [0] }, #padding value
  'filter_pool': {'values' : [3,2] }, 
  'stride_pool' : {'values' : [1,2] }, #stride value during pooling
  'padding_pool' : {'values' : [0] }, #padding value for pooling
  'activation' : {'values' : ['ReLU', 'GELU', 'SiLU', 'Mish'] }, # ReLU, GELU, SiLU, Mish,
  'dense_layer_size' : {'values' : [256,512,1024] },
  'batch_size' : {'values' : [8,32,64] },
  'image_size' : {'values' : [224,256] },
  'epochs':{'values' : [5,8,10] },
  'learning_rate' : {'values' : [0.0001,0.0003] },
  'optimizer' : {'values' : ['adam','sgd'] },
  }


metric = {
    'name' : 'Accuracy',
    'goal' : 'maximize'
}

sweep_config['metric'] = metric

sweep_config['parameters'] = parameters_sweep

In [ ]:
def train_model( settings , class_set , load_train , load_test ,load_validation ):

  def assign_optimizer(opt_type , lr , model):
    if(opt_type  == 'adam'):
      optimizer = optim.Adam(model.parameters(), lr)
      return optimizer
    if(opt_type  == 'sgd'):
      optimizer = optim.SGD(model.parameters(), lr)
      return optimizer
    if(opt_type  == 'nadam'):
      optimizer = optim.NAdam(model.parameters(), lr)
      return optimizer
  
  model = CNN(settings).to(device)
  optimizer = assign_optimizer(settings['optimizer'] , settings['learning_rate'] , model)
  criterion = nn.CrossEntropyLoss()

  for epochit in range(settings['epochs']):
      temp_loss_train = 0.0
      pred_train = 0
      pred_total = 0


      model.train()

      for images, labels in load_train:

          images = images.to(device)
          labels =  labels.to(device)

          optimizer.zero_grad()
          outputs = model.forward(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          temp_loss_train += loss.item()
          _, predicted = torch.max(outputs.data, 1)
          pred_total += labels.size(0)
          pred_train += (predicted == labels).sum().item()


      train_accuracy = pred_train / pred_total
      loss_train = temp_loss_train / len(load_train)
      print('===================================================================================')
      print('Epoch ',epochit+1, "Train Loss:", loss_train, "Train Accuracy:" ,train_accuracy)
      wandb.log({'epochs':epochit + 1, 'train_loss': loss_train, 'train_accuracy': train_accuracy})



      model.eval()
      temp_loss_val = 0.0
      pred_val = 0
      pred_total_val = 0

      with torch.no_grad():
          for val_images, val_labels in load_validation:

              val_images = val_images.to(device)
              val_labels =  val_labels.to(device)
              val_outputs = model.forward(val_images)
              val_loss = criterion(val_outputs, val_labels)
              temp_loss_val += val_loss.item()

              _, val_predicted = torch.max(val_outputs.data, 1)
              pred_total_val += val_labels.size(0)
              pred_val += (val_predicted == val_labels).sum().item()

      val_accuracy = pred_val / pred_total_val
      val_average_loss = temp_loss_val / len(load_validation)
      wandb.log({'val_loss': val_average_loss, 'val_accuracy': val_accuracy})
      print("Validation Loss:", val_average_loss, "Validation Accuracy:" ,val_accuracy)
      print('===================================================================================')


  wandb.log({'Accuracy' :val_accuracy})





In [ ]:
def main():
  param = wandb.init(project="DL_assignment_2")
  wandb.run.name = (
      ":ep-" + str(param.config.epochs) +
      ":opt-" + param.config.optimizer +
      ":a-" + param.config.activation +
      ":bs-" + str(param.config.batch_size) +
      ":fs-" + str(param.config.filter_size) +
      ":fp-" + str(param.config.filter_pool) +
      ":da-" + param.config.data_augmentation +
      ":eta-" + str(param.config.learning_rate) +
      ":dls-" + str( param.config.dense_layer_size)
  )

  settings = {
      'input_channel' : param.config.input_channel,
      'output_size' : param.config.output_size,
      'data_augmentation' : param.config.data_augmentation, # "Yes", "No"
      'batch_normalisation' : param.config.batch_normalisation, # "Yes", "No"
      'dropout' : param.config.dropout,
      'filter_organisation' : param.config.filter_organisation, #'same', 'double', 'half'
      'total_filters': param.config.total_filters, #total filters on each layer
      'filter_size' : param.config.filter_size, #each filter size
      'stride' : param.config.stride, #stride value for filter
      'padding' : param.config.padding, #padding value
      'filter_pool': param.config.filter_pool, #pooled filter size(filter_pool_size not needed)
      'stride_pool' : param.config.stride_pool, #stride value during pooling
      'padding_pool' : param.config.padding_pool, #padding value for pooling
      'activation' : param.config.activation, # ReLU, GELU, SiLU, Mish,
      'dense_layer_size' : param.config.dense_layer_size,
      'batch_size' : param.config.batch_size,
      'image_size' : param.config.image_size,
      'epochs':param.config.epochs,
      'learning_rate' : param.config.learning_rate,
      'optimizer' : param.config.optimizer,

  }
# settings = {
#   'input_channel' : 3,
#   'output_size' : 10,
#   'data_augmentation' : "No", # "Yes", "No"
#   'batch_normalization' : "No", # "Yes", "No"
#   'dropout' : 0.2,
#   'filter_organisation' : 'same', #'same', 'double', 'half'
#   'total_filters': 8, #total filters on each layer
#   'filter_size' : 3, #each filter size
#   'stride' : 1, #stride value for filter
#   'padding' : 0, #padding value
#   'filter_pool': 3, #pooled filter size(filter_pool_size not needed)
#   'stride_pool' : 1, #stride value during pooling
#   'padding_pool' : 0, #padding value for pooling
#   'activation' : 'ReLU', # ReLU, GELU, SiLU, Mish,
#   'dense_layer_size' : 16,
#   'batch_size' : 8,
#   'image_size' : 256,
#   'epochs':2,
#   'learning_rate' : 0.0001,
#   'optimizer' : 'adam',
# }
  class_set , load_train , load_test ,load_validation = dataset_loader(settings['image_size'] , settings['data_augmentation'] , settings['batch_size'], 0.125)


  train_model( settings  , class_set , load_train , load_test ,load_validation)


In [ ]:
sweep_id = wandb.sweep(sweep_config, project = 'DL_assignment_2')
wandb.agent(sweep_id, main)
wandb.finish()

In [ ]:
#pl epoch run
#validation run
#val accuracy print
#test images plotting
#sweep run
#partB